<a href="https://colab.research.google.com/github/iPoetDev/ibm-skills-ai-colab-sessions/blob/main/notebooks-labs/Session3_FineTuning_BERTandGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <ins>Session 3</ins>.2: **IBM Skills Build: Generative AI Live Technical Lab** (Part 2)

> #### **Objective**: *Understand the theory and hands-on implementation of*: <br>  1️⃣ Transformers: BERT (HuggingFace)
>> - Exploring pre-trained language models like *BERT or GPT* and
>> - Fine-tuning them for specific NLP tasks

- **URL**: [https://skills.yourlearning.ibm.com/activity/PLAN-CB1CC0D21AFB](https://skills.yourlearning.ibm.com/activity/PLAN-CB1CC0D21AFB "Programme for Artifical Intelligence: eLearning on IBM.com (Login required)")<small><sup><strong>eLearning, Login</strong></sup></small><br>
- **Share**: [Introduction to Generative AI](https://skills.yourlearning.ibm.com/activity/MDL-388 "eLearning on IBM.com (Login required") <small><sup><strong>eLearning, Login</strong></sup></small>
- **Recording**: [Recording: Live Technical Session 3](https://skills.yourlearning.ibm.com/activity/URL-6BF19B3CC379 "Video: IBM's Box.com (Login required")
- **CoLab: Source Notebook**: [https://colab.research.google.com/drive/1FW5-OGD2jegulfkF8afRptkZ3cEakL--?usp=sharing](https://colab.research.google.com/drive/1FW5-OGD2jegulfkF8afRptkZ3cEakL--?usp=sharing "Authors: Marty Bradly's Session 3 Tune Transformers")
  - Original by author: Marty Bradly: [LinkedIn](https://www.linkedin.com/in/martybradley/), [Website](https://www.evergreen-ai.com/), [GitHub @marty916](https://github.com/marty916 "Marty Bradly [July, 2024], Last accessed: July 2024")

## Steps

1.   [Setup / Install](#scrollTo=bnhqc5UADi9O&line=1&uniqifier=1)
2.   [Load Target Dataset](#scrollTo=Zl9x1VIbgs6x&line=1&uniqifier=1)
3.   [Load BERT Pretrained Model](#scrollTo=j0tWoINggxXx&line=1&uniqifier=1)
4.   [Training Arguments](#scrollTo=kQEYGBAXg2lv&line=4&uniqifier=1)
5.   [Trainer Definition](#scrollTo=2HFVqLsqg84g&line=3&uniqifier=1)
6.   [Model Training](#scrollTo=K93oznlqg_0v&line=1&uniqifier=1)
7.   [Model Evaluation](#scrollTo=yYL0MDlQhagQ&line=1&uniqifier=1)
8.   [Run Predictions](#scrollTo=zT7Ev3gRhkYK&line=3&uniqifier=1)


---
> <hr>
---

## 1. <ins>Setup / Install</ins>

- Hugging Face
    - HuggingFace's Accelerate
    - HuggingFace's Transformers
    - HuggingFace's Datasets
- PyTorch
    - Torch

In [ ]:
# Install, by upgrade, to latest version.
! pip install -U accelerate
! pip install -U transformers

In [ ]:
# Import: HuggingFace's Acclerate
import accelerate
# Version Check
print("Using accelerate", accelerate.__version__)
# Import: HuggingFace's Acclerate
import transformers
# Version Check
print("Using transformers", transformers.__version__)

> <hr>

In [ ]:
# Install, shell command, quietly
!pip install -q datasets torch
# Import
import torch
# Version Check
print("Using torch", torch.__version__)

In [ ]:
# Import BERT modules and transformers trainers
from transformers import BertTokenizer,
                        BertForSequenceClassification,
                        Trainer,
                        TrainingArguments
# Import Datasets
from datasets import load_dataset,
                     load_metric
# Version Check
print("Using datasets", datasets.__version__)

> <hr>

## 2. <ins>Load the IMDB dataset</ins>

In [ ]:
# Load the dataset
dataset = load_dataset("imdb")
tokenizer = BertTokenizer.\
                from_pretrained("bert-base-uncased")

# To convert them into numerical tokens, padding to length, truncating
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True)

# Tokenise the Dataset
tokenized_datasets = dataset.map(tokenize_function,
                                 batched=True)

> <hr>

## 3. <ins>Load BERT pretrained model</ins>

In [ ]:
# Load pre-trained BERT model
model = BertForSequenceClassification.\
            from_pretrained("bert-base-uncased",
                            num_labels=2)

> <hr>

## 4. <ins>Define the training arguments</ins>

> Defines the hyperparameters for training a model using the Hugging Face Transformers library.<br>
> - These settings control the training process,
> - Affecting the model's performance and convergence.

1. Constants for cleaner code.
2. Assign Training Args to control the training:
    - Tuning performance and convergence

In [ ]:
# Define constants for training arguments
OUTPUT_DIR = "./results"
EVAL_STRATEGY = "epoch"         # Evaluate after every epoch/pass
LEARNING_RATE = 2e-5            # Learning rate's step sizing: weights updated.
TRAIN_BATCH_SIZE = 8    	    # Forward/Backward pass training examples
EVAL_BATCH_SIZE = 8             # Forward/Backward pass evaluation examples
NUM_TRAIN_EPOCHS = 3            # Number of epochs to train
WEIGHT_DECAY = 0.01             # Regularization: prevents (penalty) overfitting

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy=EVAL_STRATEGY,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    weight_decay=WEIGHT_DECAY,
)

> <hr>

## 5. <ins>Define the trainer</ins>

> This code initializes a Trainer object from the Hugging Face Transformers library, which is used to streamline the training and evaluation of your BERT model

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,                    # the pre-trained BERT model for fine tuning
    args=training_args,             # args, inc hyperparameters.
    train_dataset=\
        tokenized_datasets["train"],    # tokenised training dataset
    eval_dataset=\
        tokenized_datasets["test"],     # tokenised test/eval dataset
)

> <hr>

## 6. <ins>Train the model</ins>
A100 - 36 min
other - 5h+


In [ ]:
# Train the model
trainer.train()

> <hr>

## 7. <ins>Evaluate the model</ins>
A100 - 4-5min

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

> <hr>

## 8. <ins>Make predictions</ins>

- Constants for cleaner code.

In [ ]:
TENSOR_FORMAT = "pt"    # PyTorch tensors
SPECIAL_DEVICE = "cuda" # GPU
DEFAULT_DEVICE = "cpu"  # CPU
STORE_DIMENSIONS = -1   # the last dimension, typically holds the class score



1.  Supply Inputs
2.  Tokenises the Inputs according to Tensor formats
3.  Move inputs / device to model
4.  Move the model for the available device
5.  Move input tensors to the same device
6.  Model predicitions
7.  Extract the predicted class label
8.  Display the predicition


In [ ]:
# Make predictions
text = "This is a great movie!"         # the input text to classify
# text = "This is a terrible movie!"    # the alt text to classify

inputs = tokenizer(                         # tokenizes: same BERT tokenizer
                    text,                   # Inputs
                    return_tensors=TENSOR_FORMAT)
                         # Output in PyTorch tenors format

# Move the inputs to the same device as the model
device = torch.device(
                    SPECIAL_DEVICE if torch.cuda.is_available()
                            else DEFAULT_DEVICE)
# Move the model for the available device
model.to(device)
# Move input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Get predictions from the model
outputs = model(**inputs)

# Extract the predicted class label (index with highest score)
predictions = torch.argmax(outputs.logits,
                           dim=STORE_DIMENSIONS)
# Display the predicition.
print(f"Predicted label: {predictions.item()}")

Predicted label: 0


---
> <center> ~ # ~ </center>
---

## Author

[![LinkedIn](https://img.shields.io/badge/Author-Charles%20J%20Fowler-0077B5?logo=gmail&logoColor=white)](mailto:ipoetdev-github-no-reply@outlook.com "Contact CJ on GItHub email: ipoetdev-github-no-reply@outlook.com") <sup>|</sup> [![LinkedIn](https://img.shields.io/badge/Charles%20J%20Fowler-LinkedIn-0077B5?logo=linkedin&logoColor=white)](https://ie.linkedin.com/in/charlesjfowler "@CharlesJFowler @Linkedin.com") <sup>|</sup> [![LinkedIn](https://img.shields.io/badge/iPoetDev-GitHub-0077B5?logo=GitHub&logoColor=white)](https://github.com/ipoetdev "@iPoetDev @GitHub")

## ChangeLog

| Date<sup>1</sup> | Version | Changed By | Change | Activity | From |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 2024-07-16  | 0.1 | Charles J Fowler  | Source uploaded | Uploaded  | [Source Notebook]( https://colab.research.google.com/drive/1FW5-OGD2jegulfkF8afRptkZ3cEakL--?usp=sharing "Author: Marty Bradly") |
| 2024-07-26  | 0.2 | Charles J Fowler  | Draft Portfolio version | Modify  | --- |  
<sup>1</sup>: `YYYY-MM-DD